## Luria Delbruck Spontaneous Mutations

In [2]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
from scipy.integrate import quad
from scipy.special import comb, factorial
from scipy.optimize import minimize
from scipy.ndimage import gaussian_filter1d
from scipy.special import betainc

np.random.seed(123)

def deterministic_growth_population(initial_population, initial_time, max_time, growth_rate):
    """
    The population of sensitive bacterial cells at the end time.
    
    Parameters:
    max_time: End time of the simulation.
    initial_population: Initial population size.
    growth_rate: Growth rate per individual.
    
    Returns:
    the total populations at the end time.
    """
    
    # sensitive cells grow exponentially
    return initial_population * np.exp(growth_rate*max_time)

def yule_process(initial_population, initial_time, max_time, growth_rate):
    """
    The growth of mutants populations 
    from initial time until end time by Yule process.
    
    Parameters:
    max_time: End time of the simulation.
    initial_population: Initial population size.
    growth_rate: Growth rate per individual.
    
    Returns:
    times: a list of time.
    population:  a list of populations.
    """
    
    time = initial_time
    population = initial_population
    times = [initial_time]
    populations = [initial_population]
    
    while time < max_time:
        
        # generate random time with rate n*cell division rate
        next_time = np.random.exponential(1 / (population * growth_rate))
        
        # population increase by 1
        time += next_time
        population += 1
        
        times.append(time)
        populations.append(population)
        
    return times, populations

def cumulative_intensity(t, initial_population, mutation_rate, growth_rate):
    """
    Parameters:
    t: Time.
    initial_population: Initial population size.
    growth_rate: Growth rate per individual.
    
    Returns:
    Cumulative intensity (integral of intensity function from 0 to t).
    """
    
    # Cumulative intensity is the integral of mutation_rate * initial_population * exp(growth_rate * u) du from 0 to t
    return (mutation_rate * initial_population / growth_rate) * (np.exp(growth_rate * t) - 1)

def generate_mutation_events(end_time, initial_population, growth_rate, mutation_rate):
    """
    Generate a list of time that mutation occurs.
    
    Parameters:
    end_time: The time at which to stop generating mutation events.
    initial_population: The initial population size.
    growth_rate: The growth rate of the population.
    mutation_rate: Mutation rate per bacterium per unit time.
    
    Returns:
    A list of mutation event times.
    """
    mutation_events = []
    current_time = 0
    
    while current_time < end_time:
        
        U = np.random.uniform(0, 1)
            
        # Solve for the next event time using the inverse transform method
        # Find time t such that cumulative_intensity(t) = cumulative_intensity(current_time) + log(1/U)
        current_intensity = cumulative_intensity(current_time, initial_population, mutation_rate, growth_rate)
        target_intensity = current_intensity + np.log(1/U)
        
        # Solve for the next time using the inverse of the cumulative intensity
        next_time = (1 / growth_rate) * np.log((growth_rate * target_intensity / (mutation_rate * initial_population)) + 1)
        
        if next_time <= end_time:
            mutation_events.append(next_time)
        
        current_time = next_time
    
    return mutation_events

def yule_process_with_mutations(end_time, initial_population, growth_rate, mutation_rate):
    """
    Mutants that occurs at set of mutation event growth with Yule process
    
    Parameters:
    end_time: The time at which to evaluate the population.
    initial_population: The initial population size.
    growth_rate: The growth rate of the population.
    mutation_rate: Mutation rate per bacterium per unit time.
    
    Returns:
    final_population: Total number of sensitve bacterial populations.
    total_mutant_population: Total number of mutant populations.
    mutation_events: A list of mutation event times.
    """
    # Calculate final population at the end time for original population
    population_at_t = deterministic_growth_population(initial_population, 0, end_time, growth_rate)
    final_population = population_at_t
    
    # Generate mutation events using the continuous-time inverse transform method
    mutation_events = generate_mutation_events(end_time, initial_population, growth_rate, mutation_rate)
    
    # Simulate the growth of mutants initiated at mutation events
    total_mutant_population = 0
    for mutation_time in mutation_events:
        
        # Calculate the mutant population at the end time using Yule process
        time_mutant, pop_mutant = yule_process(initial_population, mutation_time, end_time, growth_rate)
        mutant_final_population = pop_mutant[-1]
        total_mutant_population += mutant_final_population
    
    return final_population, total_mutant_population, mutation_events

def run_simulations(num_simulations, end_time, initial_population, growth_rate, mutation_rate):
    """
    Parameters:
    num_simulations: The number of simulations to run.
    end_time: The time at which to evaluate the population.
    initial_population: The initial population size.
    growth_rate: The growth rate of the population.
    mutation_rate: Mutation rate per bacterium per unit time.
    
    Returns:
    expected_mutant_population: Expected number of total mutant populations.
    variance_mutant_population: Variance number of total mutant populations.
    p0: Probability that no mutations have occured over time.
    expected_population: Expected number of sensitive bacterial populations.
    mutant_populations: List of number of total mutant populations for all simulations.
    final_populations: List of number of sensitive bacterial populations for all simulations.
    """
    final_populations = []
    mutant_populations = []
    mut_event = []
    no_mutation = 0
    
    for _ in tqdm(range(num_simulations), desc="Running simulations"):
        final_population, total_mutant_population, mutation_events = yule_process_with_mutations(
            end_time, initial_population, growth_rate, mutation_rate)
        
        final_populations.append(final_population)
        mutant_populations.append(total_mutant_population)
        mut_event.append(len(mutation_events))
        
        # a simulation with no mutants
        if total_mutant_population == 0:
            no_mutation += 1
    
    expected_mutant_population = np.mean(mutant_populations)
    variance_mutant_population = np.var(mutant_populations)
    p0 = no_mutation/num_simulations
    expected_population = np.mean(final_populations)
    
    return expected_mutant_population, variance_mutant_population, p0, expected_population, mutant_populations, final_populations

/Users/mig/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
if __name__ == "__main__":
    end_time = 8
    initial_population = 1
    growth_rate = 1
    mutation_rate = 0.0001
    num_simulations = 100000

    expected_mutant_population, variance_mutant_population, p0, expected_population, mutant_populations, final_populations = run_simulations(
        num_simulations, end_time, initial_population, growth_rate, mutation_rate)

print('expected mutants = ', expected_mutant_population)
print('variance of mutants = ', variance_mutant_population)


Running simulations: 100%|██████████| 100000/100000 [00:00<00:00, 128142.23it/s]

expected mutants =  2.83977
variance of mutants =  2613.7250163471003


#### Expected value and variance of the number of mutant populations from Mathematical model

In [4]:
def theory_expected_mutant(mutation_rate, growth_rate, end_time):
    """
    The expected number of mutant population from Luria and Delbruck's Legacy.
    
    Parameters:
    end_time: The time at which to stop generating mutation events.
    growth_rate: The growth rate of the population.
    mutation_rate: Mutation rate per bacterium per unit time.
    
    Returns:
    Expected number of mutant population from Luria and Delbruck's Legacy.
    """
    return mutation_rate * end_time * np.exp(growth_rate * end_time)

def theory_var_mutant(mutation_rate, growth_rate, end_time):
    """
    The variance of mutant population from Luria and Delbruck's Legacy.
    
    Parameters:
    end_time: The time at which to stop generating mutation events.
    growth_rate: The growth rate of the population.
    mutation_rate: Mutation rate per bacterium per unit time.
    
    Returns:
    Variance of mutant population from Luria and Delbruck's Legacy.
    """
    return (((mutation_rate * np.exp(growth_rate * end_time)) / growth_rate) * (2 * np.exp(growth_rate * end_time) - growth_rate * end_time - 2))

In [5]:
print('expected mutants = ', theory_expected_mutant(mutation_rate, growth_rate, end_time))
print('variance of mutants = ', theory_var_mutant(mutation_rate, growth_rate, end_time))

expected mutants =  2.384766389633383
variance of mutants =  1774.241146114533


### Estimating mutation rate

#### poisson approximation method ($p_0$)

In [6]:
def mutation_rate_p0(growth_rate, p0, expected_population):
    """
    Estimating mutation rate from poisson approximation (p0)
    
    Parameters:
    p0: fraction of number of cultures with no mutation.
    growth_rate: The growth rate of the population.
    expected_population: Total population for bacteria.
    
    Return: mutation rate from poisson approximation (p0)
    """
    return -growth_rate*np.log(p0)/expected_population

In [7]:
print('Estimating mutation rate from poisson approximation (p0) = ', mutation_rate_p0(growth_rate, p0, expected_population))

Estimating mutation rate from poisson approximation (p0) =  0.00010045690109231654


#### Drake's equation

In [8]:
def mutation_rate_drake_equation(expected_population, mutant_population, growth_rate):
    """
    Estimating mutation rate from Drake's equation
    
    Parameters:
    expected_population: Total population for bacteria.
    mutant_population: Total population of mutants.
    growth_rate: The growth rate of the population.
    
    Return: mutation rate from Drake's equation
    """
    
    # the fraction of mutants and the sensitive cells
    f = np.mean(mutant_population)/expected_population
    
    return growth_rate*f/np.log(np.mean(expected_population))

In [9]:
print('Estimating mutation rate from Drakes equation = ', mutation_rate_drake_equation(expected_population, expected_mutant_population, growth_rate))

Estimating mutation rate from Drakes equation =  0.00011907958835483951


#### Maximum Likelihood Method - Proportion of Cultures with Mutants

In [10]:
def mutation_rate_ML1(expected_population, mutant_population, num_simulations):
    """
    Estimating mutation rate from Maximum Likelihood Method 
    Proportion of Cultures with Mutants
    
    Parameters:
    expected_population: Total population for bacteria.
    mutant_population: Total population of mutants.
    num_simulations: Number of simulations.
    
    Return: mutation rate from Maximum Likelihood Method 
    Proportion of Cultures with Mutants
    """
    X = 0
    # count number of simultaions with no mutation
    for k in mutant_population:
        if k == 0:
            X += 1
    N = expected_population
    C = num_simulations

    return -np.log(X/C)/N

In [11]:
print('Estimating mutation rate from Maximum Likelihood Method - Proportion of Cultures with Mutants = ', mutation_rate_ML1(expected_population, mutant_populations, num_simulations))

Estimating mutation rate from Maximum Likelihood Method - Proportion of Cultures with Mutants =  0.00010045690109231654


#### Maximum Likelihood Method - Full Mutant Distribution 


In [16]:
def pmf_spec_yule(end_time, initial_population, growth_rate, count):
    """
    The probability mass function (PMF) 
    for the Yule process
    
    Parameters:
    end_time: Total time for population growth
    initial_population: Starting population size
    growth_rate: Growth rate of the population
    count: Number of individuals in the population
    """
    return np.exp(growth_rate * end_time) * (1 - betainc(2, count, np.exp(- growth_rate * end_time)))/(count*(count+1)*(np.exp(growth_rate * end_time) - 1))
def pmf(end_time, initial_population, mu, growth_rate):
    """
    Compute the PMF using cumulative intensity
    
    Parameters:
    end_time: Total time for population growth  
    initial_population: Starting population size  
    mu: Mutation rate  
    growth_rate: Growth rate of the population  

    Returns:
    pmf_func: Probability mass function (PMF) for mutant population size
    """
    
    # Cumulative intensity function
    ci = cumulative_intensity(end_time, initial_population, mu, growth_rate)
    pmf_func = np.zeros(2000)
    pmf_func1 = np.zeros(2000)

    m = - ci
    pmf_func[0] = math.exp(m)
    for k in range(1, 2000):
        sum = 0
        for i in range(0, k):
            sum += ci * (k - i) / k * pmf_func[i] * pmf_spec_yule(end_time, initial_population, growth_rate, k - i)
        pmf_func[k] = sum
    return pmf_func

def log_likelihood(mu, data, end_time, initial_population, growth_rate, h):
    """
    Compute the log-likelihood for given mutation rate
    
    Parameters:
    mu: Mutation rate  
    data: Observed mutant population sizes  
    end_time: Total time for population growth  
    initial_population: Starting population size  
    growth_rate: Growth rate of the population  
    h: Histogram of mutant population counts  
   
    Returns:
    Negative log-likelihood for optimization  
    """

    pmf_func = pmf(end_time, initial_population,mu, growth_rate)
    pmf_func = np.maximum(pmf_func, 1e-10)
    likelihood = 0
    for k in data:
        if k < len(pmf_func):
            likelihood += h[k] * np.log(pmf_func[k] + 1e-10)  # Add epsilon to avoid log(0)
        else:
            likelihood += -np.inf  # Assign very low likelihood for out-of-range data
    print(-likelihood)
    return -likelihood  # Return negative for minimization

mutation_rate = [10 ** -4]
num_simulations = 5000

mut_rate = np.zeros(len(mutation_rate))

for i in range(len(mutation_rate)):
    # run simulation for each mutation rate
    expected_mutant_population, variance_mutant_population, p0, expected_population, mutant_populations, final_populations = run_simulations(
        num_simulations, end_time, initial_population, growth_rate, mutation_rate[i])
    h = np.zeros(2000)
    
    for j in range(num_simulations):
        for q in range(len(h)):
            if q == mutant_populations[j]:
                h[q] += 1
    h_smooth = h[:2000] * num_simulations
  
    freq_m = gaussian_filter1d(h_smooth, sigma=0.45)

    data = range(2000)
    
    # Choose initial guess
    mu_initial = 10 ** (-5)

    # Perform MLE using scipy.optimize.minimize
    result = minimize(
       lambda log_mu: log_likelihood(np.exp(log_mu), data, end_time, initial_population, growth_rate, freq_m),
       x0=[np.log(mu_initial)],
       bounds=[(np.log(1e-10), np.log(1))],  
       method='L-BFGS-B'
       )
    mut_rate[i] = np.exp(result.x[0])


Running simulations: 100%|██████████████| 5000/5000 [00:00<00:00, 124054.40it/s]
/var/folders/s3/bs4t_w8j12j88rywpg85yqr40000gn/T/ipykernel_19937/1349006584.py:33: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pmf_func[0] = math.exp(m)
/var/folders/s3/bs4t_w8j12j88rywpg85yqr40000gn/T/ipykernel_19937/1349006584.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pmf_func[k] = sum


48158202.76535891
48158202.69438148
558317593.7345111
558317593.7345111
46163860.45172327
46163860.382620335
243708237.3958877
243708239.85236448
36518499.10233608
36518499.06807292
46203794.07320594
46203794.28890625
35450034.64772746
35450034.64236275
35433174.5094279
35433174.51132834
35430932.60735166
35430932.60728166
35430929.50962439
35430929.509623565
35430929.509153455
35430929.50915343


In [17]:
print('Estimating mutation rate from Maximum Likelihood Method - Full Mutant Distribution = ', mut_rate[0])

Estimating mutation rate from Maximum Likelihood Method - Full Mutant Distribution =  0.00013047757950711714
